In [3]:
import os
import json
import pandas as pd
import traceback


In [9]:
from langchain_openai import ChatOpenAI


In [10]:
from dotenv import load_dotenv
load_dotenv()   #take environment variables from .env 

True

In [11]:
KEY=os.getenv("OPENAI_API_KEY")

In [13]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.3)

In [14]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000138432EB7C0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000138432F5F40>, temperature=0.3, openai_api_key=SecretStr('**********'), openai_proxy='')

In [22]:
from langchain_openai.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [23]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [24]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [25]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [29]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

c:\Users\seshareddy\mcqgen\env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [26]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [54]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [55]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [56]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [58]:
file_path=r"C:\Users\seshareddy\mcqgen\data.txt"

In [59]:
file_path

'C:\\Users\\seshareddy\\mcqgen\\data.txt'

In [60]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [61]:
print(TEXT)

Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processes, scientific visualization, algorithms and systems to extract or extrapolate knowledge and insights from potentially noisy, structured, or unstructured data.[2]

Data science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]

Data science is "a concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge.[6] However, data science is different from computer science and information scien

In [62]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [63]:
NUMBER=8 
SUBJECT="DataScience"
TONE="medium"

In [64]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processes, scientific visualization, algorithms and systems to extract or extrapolate knowledge and insights from potentially noisy, structured, or unstructured data.[2]

Data science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]

Data science is "a concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information scie

In [65]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2747
Prompt Tokens:2020
Completion Tokens:727
Total Cost:0.0021005


In [66]:
quiz=response.get("quiz")

In [67]:
quiz=json.loads(quiz)

In [68]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [69]:
quiz_table_data

[{'MCQ': 'What is data science primarily focused on?',
  'Choices': 'a: Extracting knowledge from large data sets | b: Creating programming code | c: Developing software applications | d: Designing user interfaces',
  'Correct': 'a'},
 {'MCQ': 'Who is considered a data scientist?',
  'Choices': 'a: A professional who designs websites | b: A professional who creates insights from data using statistical knowledge | c: A professional who works in customer service | d: A professional who specializes in graphic design',
  'Correct': 'b'},
 {'MCQ': 'What did John Tukey describe in 1962 that resembles modern data science?',
  'Choices': 'a: Data visualization techniques | b: Data analysis field | c: Scientific computing methods | d: Statistical modeling approaches',
  'Correct': 'b'},
 {'MCQ': 'What term did C. F. Jeff Wu use for the first time in 1985 as an alternative name for statistics?',
  'Choices': 'a: Data analysis | b: Data mining | c: Data science | d: Computer science',
  'Correct'

In [70]:
quiz=pd.DataFrame(quiz_table_data)

In [72]:
quiz

,MCQ,Choices,Correct
0,What is data science primarily focused on?,a: Extracting knowledge from large data sets |...,a
1,Who is considered a data scientist?,a: A professional who designs websites | b: A ...,b
2,What did John Tukey describe in 1962 that rese...,a: Data visualization techniques | b: Data ana...,b
3,What term did C. F. Jeff Wu use for the first ...,a: Data analysis | b: Data mining | c: Data sc...,c
4,"According to Vasant Dhar, what does statistics...",a: Prediction and action | b: Quantitative dat...,b
5,What did Hayashi Chikio argue for in 1998 as a...,a: Data visualization | b: Data analysis | c: ...,c
6,What were popular terms for finding patterns i...,a: Data visualization | b: Knowledge discovery...,b
7,What did the American Statistical Association ...,"a: Database management, statistics and machine...",a


In [71]:
quiz.to_csv("DataScience2.csv",index=False)